In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, roc_auc_score,log_loss
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import Pipeline
import os
from tqdm import tqdm
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression 
from sklearn.neighbors import KNeighborsClassifier


import warnings
warnings.filterwarnings('ignore')


In [41]:
train = pd.read_csv('train.csv')

In [42]:
X_train = train.drop('price',axis=1).set_index('id')
y_train = train['price']

In [43]:
#Linear Regression

In [44]:
ohe = OneHotEncoder(handle_unknown = 'infrequent_if_exist',
                   sparse_output= False,
                    min_frequency = 10000).set_output(transform='pandas')

In [45]:
ct_ohe = make_column_transformer(('passthrough',make_column_selector(dtype_exclude=object)),
                             (ohe,make_column_selector(dtype_include=object)),
                             verbose_feature_names_out=False).set_output(transform='pandas')

In [46]:
X_train_ohe = ct_ohe.fit_transform(X_train)

In [47]:
X_train_ohe.isnull().sum().sum()

0

In [48]:
X_test  = pd.read_csv('test.csv',index_col = 0)

In [49]:
X_test_ohe = ct_ohe.transform(X_test)

In [50]:
len(X_test_ohe.columns)

36

In [51]:
len(X_train_ohe.columns)

36

In [52]:
lr = LinearRegression()

In [53]:
lr.fit(X_train_ohe,y_train)

LinearRegression()

In [54]:
y_pred = lr.predict(X_test_ohe)

In [55]:
len(y_pred[(y_pred<150) & (y_pred>0)])

76

In [56]:
np.quantile(y_pred,[0,0.25,0.50,0.75,1])

array([-81470.85744057,  27200.53164406,  46874.64574772,  62937.89171714,
       108722.97041331])

In [57]:
y_train.min()

2000

In [58]:
y_pred[y_pred < 150] = 150

In [59]:
np.quantile(y_pred,[0,0.25,0.50,0.75,1])

array([   150.        ,  27200.53164406,  46874.64574772,  62937.89171714,
       108722.97041331])

In [60]:
y_pred

array([24081.99096333, 66756.81997001, 68790.37275497, ...,
       28728.91869523, 17861.98420665, 40548.27085718])

In [61]:
submit  = pd.read_csv('sample_submission.csv')

In [62]:
submit['price'] = y_pred

In [63]:
submit.to_csv('ss_5_nov_1_lr.csv',index=False)

In [64]:
#ElasticNet

In [65]:
#X_train, X_test,y_train,y_test = train_test_split(X_train,y_train,random_state=24,test_size=0.3)
                                                 
elastic = ElasticNet()
kfold= KFold(n_splits=5, shuffle= True, random_state=24)
pipe= Pipeline([('CT',ct_ohe),('EL',elastic)])
params= {'EL__alpha':np.linspace(0.001,5,3), 'EL__l1_ratio':np.linspace(0.0001,1,3)}
gcv_el = GridSearchCV(pipe, param_grid=params, verbose=3, cv=kfold)
gcv_el.fit(X_train,y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END EL__alpha=0.001, EL__l1_ratio=0.0001;, score=0.088 total time=   5.0s
[CV 2/5] END EL__alpha=0.001, EL__l1_ratio=0.0001;, score=0.076 total time=   5.4s
[CV 3/5] END EL__alpha=0.001, EL__l1_ratio=0.0001;, score=0.085 total time=   4.9s
[CV 4/5] END EL__alpha=0.001, EL__l1_ratio=0.0001;, score=0.106 total time=   4.8s
[CV 5/5] END EL__alpha=0.001, EL__l1_ratio=0.0001;, score=0.086 total time=   5.4s
[CV 1/5] END EL__alpha=0.001, EL__l1_ratio=0.50005;, score=0.088 total time=   4.6s
[CV 2/5] END EL__alpha=0.001, EL__l1_ratio=0.50005;, score=0.076 total time=   4.5s
[CV 3/5] END EL__alpha=0.001, EL__l1_ratio=0.50005;, score=0.085 total time=   5.1s
[CV 4/5] END EL__alpha=0.001, EL__l1_ratio=0.50005;, score=0.106 total time=   4.5s
[CV 5/5] END EL__alpha=0.001, EL__l1_ratio=0.50005;, score=0.086 total time=   4.2s
[CV 1/5] END .EL__alpha=0.001, EL__l1_ratio=1.0;, score=0.088 total time=   5.7s
[CV 2/5] END .EL__alpha=

GridSearchCV(cv=KFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000019F77056840>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                                       min_frequency=10000,
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000019F77056300>)],
                                                          verbose_feature_names_out=False)),
                                       ('EL', ElasticNet())]),
             param_grid={'EL__alpha': array([1.0000e-03, 2.5005e+00, 5.0000e+00]),
                         'EL__l1_ratio': array([1.0000e-04, 5.0005e-01, 1.0000e+00])},
             verbose=3)

In [66]:
print("Best Params:", gcv_el.best_params_)
print("Best Score:", gcv_el.best_score_)


Best Params: {'EL__alpha': 0.001, 'EL__l1_ratio': 0.0001}
Best Score: 0.08826353886702507


In [70]:
bm_el = gcv_el.best_estimator_
y_pred = bm_el.predict(X_test)

In [71]:
y_pred[y_pred < 150] = 150

In [72]:
y_pred

array([24133.22484711, 66754.40936485, 68793.04806449, ...,
       28854.88001739, 17973.80767985, 40608.5309586 ])

In [73]:
submit  = pd.read_csv('sample_submission.csv')
submit['price'] = y_pred
submit.to_csv('ss_5_nov_1_lr.csv',index=False)

In [ ]:
#K-NN Regresssor

In [ ]:
scaler_mm = MinMaxScaler()
scaler_std = StandardScaler()
ct = make_column_transformer(('passthrough', make_column_selector(dtype_exclude=object)  ),
                             (ohe, make_column_selector(dtype_include=object) ),
                            verbose_feature_names_out=False).set_output(transform='pandas')

In [82]:
knn = KNeighborsRegressor()

pipe = Pipeline([('CT',ct_ohe),('SCL',None),('KNN',knn)])
kfold = KFold(n_splits=5, random_state=24,shuffle=True)
params = {'KNN__n_neighbors': np.arange(1,8),
          'SCL':[scaler_mm, scaler_std, None]}
gcv = GridSearchCV(pipe, param_grid=params,scoring='r2',cv=kfold, verbose=3)

NameError: name 'KNeighborsRegressor' is not defined

In [79]:
bm_knn = gcv.best_estimator_
y_pred = bm_knn.predict(X_test)

NameError: name 'gcv' is not defined

In [80]:
y_pred[y_pred<0]

array([], dtype=float64)

In [81]:
y_pred[y_pred<150] = 150